In [ ]:
import os
from pprint import pprint

def count_relations(folder_path):
    # Create a dictionary to store the counts
    counts = {}
    # Loop through all subdirectories
    for subdir, _, files in os.walk(folder_path):
        # print(subdir)
        # Extract the code from the subdirectory name
        full_folder_name = os.path.basename(subdir).split('.')
        code = full_folder_name[0]
        # code = full_folder_name[1]
        # print(code)
        if len(full_folder_name) == 1:
            continue
        # Skip any subdirectories that don't contain a .conll file
        # if not any(f.endswith('.conll') for f in files):
        #     continue

        # print(full_folder_name[1])


        # Read the CURATION_USER.tsv file, if it exists
        user_file = os.path.join(subdir, 'CURATION_USER.tsv')
        # print("user_file ", os.path.exists(user_file), user_file)
        if not os.path.exists(user_file):
            continue
        

        with open(user_file, 'r', encoding='utf-8') as f:
            # Initialize the counts for this code
            code_counts = {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 0}
            # Loop through each line in the file
            for line in f:
                # Split the line into columns
                # columns = line.strip().split('\t')
                # # Extract the relation type from the third column
                # relation = columns[2]
                # Increment the count for this relation type, if it's one of the three we're counting
                for relation , number in code_counts.items():
                    n = line.count(relation)
                    code_counts[relation] += n
            # Store the counts for this code in the main dictionary
            counts[code] = code_counts
    return counts
dev_data_raw = "/Users/n2t2k/Documents/Studying/Master/Thesis/InProgress/Coding/ORIGIN_RUN_ALL_edge-oriented-graph-master-studying/dataProcessingOfficialCleaned/VLSP2020_RE_dev_org_use_TRAIN_as_DEV"
dev_raw_dict = count_relations(dev_data_raw)

pprint(dev_raw_dict)

In [ ]:
import os

def count_relations_in_folder(folder_path, relation_types):
    counts = {}
    for filename in os.listdir(folder_path):
        code_counts = {'LOCATED': 0, 'AFFILIATION': 0, 'PART - WHOLE': 0}
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            code = filename.split('.')[0]
            # print(file_path)
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    for relation , number in code_counts.items():
                        n = line.count(relation)
                        code_counts[relation] += n
            counts[code] = code_counts

    return counts

dev_data_pubtabtor =  "/Users/n2t2k/Documents/Studying/Master/Thesis/InProgress/Coding/ORIGIN_RUN_ALL_edge-oriented-graph-master-studying/dataProcessingOfficialCleaned/dev_processed/processed_txt"
rel = ['LOCATED', 'AFFILIATION', 'PART - WHOLE'] 
dev_data_pubtabtor_dict = count_relations_in_folder(dev_data_pubtabtor, rel)
pprint(dev_data_pubtabtor_dict)

In [ ]:
def subtract_dicts(d1, d2):
    result = {}
    for code in d1:
        if code not in d2:
            result[code] = d1[code]
        else:
            result[code] = {}
            for t in d1[code]:
                result[code][t] = d1[code][t] - d2[code].get(t, 0)
    return result

dev_data_dict_subtract = subtract_dicts(dev_data_pubtabtor_dict, dev_raw_dict)
print("dev_data_dict_subtract >>> ")
# pprint(dev_data_dict_subtract)
i = 0
for code, _count in dev_data_dict_subtract.items():
    if dev_data_pubtabtor_dict.get(code, None) and dev_raw_dict.get(code, None):
        if _count['AFFILIATION'] or _count['LOCATED'] or _count['PART - WHOLE']:
            i+=1
            print(f"{i}: {code}, {_count}")
